In [2]:
"""
============================
Plotting NEAT Classifier
============================

An example plot of :class:`neuro_evolution._neat.NEATClassifier`
"""
from matplotlib import pyplot as plt
# from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from neuro_evolution import NEATClassifier
from keras.datasets import cifar10
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.utils import to_categorical
import numpy as np
import cv2

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

#Transform (50000, 32, 32, 3) to (50000, 32, 32) with grayscaling.
# x_train = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in x_train])
# x_test = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in x_test])

#Preprocess the data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
# y_train = to_categorical(y_train, 10)
# y_test = to_categorical(y_test, 10)

# one_hot_encoder = OneHotEncoder(sparse=False)

# one_hot_encoder.fit(y_train)

# y_train = one_hot_encoder.transform(y_train)
# y_test = one_hot_encoder.transform(y_test)

# def testSet(x,y,val):
#     X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=val, random_state=42)
#     return X_test,Y_test

# x_test,y_test = testSet(x_test,y_test, 0.001)
# x_train,y_train = testSet(x_train,y_train, 0.001)

print(x_test.shape)
print(y_test.shape)

x_train_fl = x_train.reshape((x_train.shape[0], -1))
x_test_fl = x_test.reshape((x_test.shape[0], -1))

(10000, 32, 32, 3)
(10000, 1)


In [11]:
clf = NEATClassifier(number_of_generations=150,
                     fitness_threshold=0.90,
                     pop_size=150)

In [ ]:
neat_genome = clf.fit(x_train_fl, y_train.ravel())
y_predicted = neat_genome.predict(x_test_fl)

fig = plt.figure()
ax = plt.axes(projection='3d')

#Data for three-dimensional scattered points
train_z_data = x_train_fl
train_x_data = x_train_fl[:, 1]
train_y_data = x_train_fl[:, 0]
ax.scatter3D(train_x_data, train_y_data, train_z_data, c='Blue')

test_z_data = y_predicted
test_x_data = x_test_fl[:, 1]
test_y_data = x_test_fl[:, 0]
ax.scatter3D(test_x_data, test_y_data, test_z_data, c='Red')
ax.legend(['Actual', 'Predicted'])
plt.show()

print(classification_report(y_test.ravel(), y_predicted.ravel()))

In [7]:
pip freeze

absl-py==0.12.0
anyio==2.2.0
argon2-cffi==20.1.0
astor==0.8.1
astunparse==1.6.3
async-generator==1.10
attrs==21.2.0
Babel==2.9.1
backcall==0.2.0
bleach==3.3.0
cached-property==1.5.2
cachetools==4.2.2
certifi==2020.12.5
cffi==1.14.5
chardet==4.0.0
contextvars==2.4
cycler==0.10.0
dataclasses==0.8
decorator==5.0.7
defusedxml==0.7.1
entrypoints==0.3
flatbuffers==1.12
gast==0.4.0
google-auth==1.30.0
google-auth-oauthlib==0.4.4
google-pasta==0.2.0
grpcio==1.34.1
h5py==3.1.0
idna==2.10
immutables==0.15
importlib-metadata==4.0.1
ipykernel==5.5.4
ipython==7.16.1
ipython-genutils==0.2.0
ipywidgets==7.6.3
jedi==0.18.0
Jinja2==2.11.3
joblib==1.0.1
json5==0.9.5
jsonschema==3.2.0
jupyter==1.0.0
jupyter-client==6.1.12
jupyter-console==6.4.0
jupyter-core==4.7.1
jupyter-server==1.7.0
jupyterlab==3.0.15
jupyterlab-pygments==0.1.2
jupyterlab-server==2.5.1
jupyterlab-widgets==1.0.0
Keras==2.4.3
Keras-Applications==1.0.8
keras-nightly==2.5.0.dev2021032900
Keras-Preprocessing==1.1.2
kiwisolver==1.3.1
Markdo